# 第六讲 循环神经网络

## 一，循环神经网络（Recurrent Neural Network，RNN）

### 1，卷积神经网络与循环神经网络简单对比

### 2，详解 RNN

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, SimpleRNN
from matplotlib import pyplot as plt
import os 

input_word = 'abcde'
w_to_id = {'a':0,'b':1, 'c':2, 'd':3, 'e':4} # 单词映射到数值id的词典
id_to_onehot = {0:[1., 0, 0, 0, 0],
                1:[0, 1., 0, 0, 0],
                2:[0, 0, 1., 0, 0],
                3:[0, 0, 0, 1., 0],
                4:[0, 0, 0, 0, 1.],
                }
x_train = [id_to_onehot[w_to_id['a']],
           id_to_onehot[w_to_id['b']],
           id_to_onehot[w_to_id['c']],
           id_to_onehot[w_to_id['d']],
           id_to_onehot[w_to_id['e']],
           ]

y_train = [w_to_id['b'],
           w_to_id['c'],
           w_to_id['d'],
           w_to_id['e'],
           w_to_id['a'],
           ]

np.random.seed(7)
np.random.shuffle(x_train)
np.random.seed(7)
np.random.shuffle(y_train)
tf.random.set_seed(7)

# 使x_train符合SimpleRNN输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]。
# 此处整个数据集送入，送入样本数为len(x_train)；输入1个字母出结果，循环核时间展开步数为1; 表示为独热码有5个输入特征，每个时间步输入特征个数为5
x_train = np.reshape(x_train, (len(x_train), 1, 5))
y_train = np.array(y_train)

model = tf.keras.Sequential([
    SimpleRNN(3),
    Dense(5, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])
checkpoint_save_path = './checkpoint/rnn_onehot_1pre1.ckpt'

if os.path.exists(checkpoint_save_path + '.index'):
    print('-----------load the model -------')
    model.load_weights(checkpoint_save_path)
    
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True, 
                                                 save_best_only=True, 
                                                 monitor='loss')
history = model.fit(x_train, y_train, batch_size=32, epochs=100, callbacks=[cp_callback])
model.summary()

# print(model.trainable_variables)
file = open('./weights.txt', 'w')  # 参数提取
for v in model.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()

###############################################    show   ###############################################

# 显示训练集和验证集的acc和loss曲线
acc = history.history['sparse_categorical_accuracy']
loss = history.history['loss']

# plt.subplot(1, 2, 1)
# plt.plot(acc, label='Training Accuracy')
# plt.title('Training Accuracy')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(loss, label='Training Loss')
# plt.title('Training Loss')
# plt.legend()
# plt.show()

-----------load the model -------
Epoch 1/100
1/1 [==============================] - 1s 936ms/step - loss: 0.4219 - sparse_categorical_accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 141ms/step - loss: 0.4136 - sparse_categorical_accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 122ms/step - loss: 0.4055 - sparse_categorical_accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 132ms/step - loss: 0.3974 - sparse_categorical_accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 123ms/step - loss: 0.3895 - sparse_categorical_accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 124ms/step - loss: 0.3818 - sparse_categorical_accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 131ms/step - loss: 0.3741 - sparse_categorical_accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 124ms/step - loss: 0.3666 - sparse_categorical_accuracy: 1.0000
Epoch 9/100
1/1 [=====

In [2]:
############### predict #############
preNum = int(input('input the number of test alphabet:'))
for i in range(preNum):
    alphabet1 = input('input test alphabet')
    alphabet = [id_to_onehot[w_to_id[alphabet1]]]
    # 使alphabet符合SimpleRNN输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]。此处验证效果送入了1个样本，送入样本数为1；输入1个字母出结果，所以循环核时间展开步数为1; 表示为独热码有5个输入特征，每个时间步输入特征个数为5
    alphabet = np.reshape(alphabet, (1, 1, 5))
    result = model.predict([alphabet])
    pred = tf.argmax(result, axis=1)
    pred = int(pred)
    tf.print(alphabet1 + '->' + input_word[pred])

1/1 [==============================] - 0s 145ms/step
c->d


## 二，LSTM、GRU

### 1，原始 RNN 的问题

### 2，LSTM

### 3，GRU